<a href="https://colab.research.google.com/github/shasso2s/pytorch/blob/main/Exercise_Filter_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Idea:** 

low_from_directory() verwendet die Verzeichnisstruktur, um Dateien zu finden und zu lesen. Sie liest manuell alle Dateien aus dem Verzeichnis und speichert Dateinamen und Klassen in einem Python-Generator. Während des Trainings/der Inferenz wird der Generator aufgerufen, um die Dateinamen abzurufen, und dann werden die entsprechenden Bilder gelesen, ergänzt und für das Training/die Inferenz verwendet. Eine andere Idee ist es, bei der Erstellung des Generators in die Tiefe zu gehen (siehe Ref. 4) und nur die gefilterten Bilder an den Generator zu übergeben. Dies sollte langsam sein, aber keinen Speicherplatz beanspruchen. Diese Änderung wird auch nur einmal während des train_image_generator ausgeführt.

Die Idee hinter dieser Implementierung ist es, tief in den Generator hineinzugehen (ref 4) und nur die gefilterten Bilder an den Generator zu übergeben. Dies sollte langsam sein, aber keinen Speicherplatz beanspruchen. Diese Änderung wird auch nur einmal während des train_image_generator ausgeführt.










References:

1- https://vijayabhaskar96.medium.comtutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

2- https://github.com/keras-team/keras/blob/v2.7.0/keras/preprocessing/image.py#L898-L992

3- https://github.com/keras-team/keras/blob/v2.7.0/keras/preprocessing/image.py#L323-L410

4- https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/image/directory_iterator.py

5- https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d


1.   **Filter** function nimmt als parameter **images_filename**
2.   die Function gibt einen **booleanwert** zurück basierend auf **some processing()**
1.  some_processing() implementiert den Filter wie vom Benutzer gewünscht gibt **1** zurück, wenn der Filter für die Bilddatei **erfolgreich** war und **0**, wenn der Filter für die Bilddatei **fehlschlägt**

In [ ]:
def filter(image_filename):
    boolvalue = some_processing(image_filename)
    return boolvalue



1.   **flow_from_directory()** ruft **DictionaryIterator.init()**,diese liest directories  und ruft files und classes ab
2.  **In the init()**, wir wenden filter an , hier werden nur selected imges zugelassen

1.   **flow_from_directory()** ruft DirectoryIterator klass konstructor, und gibt filter as parameter


1.  **Note**:
filter: nimmt  der Dateinamen auf und  für jeden Dateinamen gibt einen booelan wert entsprechend dem Filter zurück
 








In [ ]:
def flow_from_directory(self,
                          directory,
                          filter,
                          target_size=(256, 256),
                          color_mode='rgb',
                          classes=None,
                          class_mode='categorical',
                          batch_size=32,
                          shuffle=True,
                          seed=None,
                          save_to_dir=None,
                          save_prefix='',
                          save_format='png',
                          follow_links=False,
                          subset=None,
                          interpolation='nearest'
                          ):

    return DirectoryIterator(
        directory,
        filter,
        self,
        target_size=target_size,
        color_mode=color_mode,
        classes=classes,
        class_mode=class_mode,
        data_format=self.data_format,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=seed,
        save_to_dir=save_to_dir,
        save_prefix=save_prefix,
        save_format=save_format,
        follow_links=follow_links,
        subset=subset,
        interpolation=interpolation)

Given below is the class declaration for DirectoryIterator, we have passed on the filter parameter to the constructor. The exact definition of __init__() is given in the cell after this.

Original Code:https://github.com/keras-team/keras/blob/v2.7.0/keras/preprocessing/image.py#L323-L410
Changes: Lines 2, 28 only


In [ ]:
class DirectoryIterator(image.DirectoryIterator, Iterator):  # pylint: disable=inconsistent-mro
    def __init__(self, directory, filter, image_data_generator,
               target_size=(256, 256),
               color_mode='rgb',
               classes=None,
               class_mode='categorical',
               batch_size=32,
               shuffle=True,
               seed=None,
               data_format=None,
               save_to_dir=None,
               save_prefix='',
               save_format='png',
               follow_links=False,
               subset=None,
               interpolation='nearest',
               dtype=None):
               
  if data_format is None:
      data_format = backend.image_data_format()
    kwargs = {}
  if 'dtype' in tf_inspect.getfullargspec(
        image.ImageDataGenerator.__init__)[0]:
  if dtype is None:
        dtype = backend.floatx()
      kwargs['dtype'] = dtype
    super(DirectoryIterator, self).__init__(
        directory, filter, image_data_generator, 
        target_size=target_size,
        color_mode=color_mode,
        classes=classes,
        class_mode=class_mode,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=seed,
        data_format=data_format,
        save_to_dir=save_to_dir,
        save_prefix=save_prefix,
        save_format=save_format,
        follow_links=follow_links,
        subset=subset,
        interpolation=interpolation,
        **kwargs)

"class DirectoryIterator(image.DirectoryIterator, Iterator):  # pylint: disable=inconsistent-mro\n    def __init__(self, directory, filter, image_data_generator,\n               target_size=(256, 256),\n               color_mode='rgb',\n               classes=None,\n               class_mode='categorical',\n               batch_size=32,\n               shuffle=True,\n               seed=None,\n               data_format=None,\n               save_to_dir=None,\n               save_prefix='',\n               save_format='png',\n               follow_links=False,\n               subset=None,\n               interpolation='nearest',\n               dtype=None):\n               \n  if data_format is None:\n      data_format = backend.image_data_format()\n    kwargs = {}\n  if 'dtype' in tf_inspect.getfullargspec(\n        image.ImageDataGenerator.__init__)[0]:\n  if dtype is None:\n        dtype = backend.floatx()\n      kwargs['dtype'] = dtype\n    super(DirectoryIterator, self).__init__(\

Nachstehend finden Sie die genaue init()-Definition, die die Verarbeitung durchführt. Wir wenden den Filter() auf die Dateinamen an und leiten nur die Dateien weiter, die den Filter() bestehen. 

Die Dateinamen und Klassen werden mit der Funktion _list_valid_filenames_in_directory() 


**Original Code**: https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/image/directory_iterator.py Changes:

In [ ]:

import multiprocessing.pool
import os
import numpy as np
from .iterator import BatchFromFilesMixin, Iterator
from .utils import _list_valid_filenames_in_directory


class DirectoryIterator(BatchFromFilesMixin, Iterator):

  """ Argument:
              Iterator: liest images aus einer Directory auf disk
              Directory : path to diretory um die Bilder zu lesen
              image_data_generator: instance von ImageDataGenerator, bentzt für random transformation and normalization
              target_size:tuple von integers , für resizing input images
              color_mode: "rgb","rgba","grayscale"
              classes: optinal list von strings , enthältet name von subdirectories zum beispiel["Dog","Cat"]
              class_mode: "binary" wenn es nur  zwei classes gibt
                           "categorical": categorical targets,
                           "sparse" :integer target
                           "input": targets sind images identish to input images
                           "None" es gibt kein target image
              batch_size: size von batch
              shulle: shulle is boolean wert zeigt ob die dataset kann randomly shuffle or not
              seed: random seed für shuffling
              data_format: string" channels_firts" , channels_last"
              save_to_dir_ : diretory für images speichern
              save prefix: ist ein string für die gespeicherte directory
              save_format: welche format muss der direc gespeichert werden.
              follow_links: boolean , links to subdirct
              subset: subset von data[train, val]
              interpolation: benutzt für resample wenn die target zise ist anders als loaded image
              keep_aspect_ratio: boolean , resize images to target size
      """"
   pe: Dtype to use for generated arrays.

    allowed_class_modes = {'categorical', 'binary', 'sparse', 'input', None}

    def __new__(cls, *args, **kwargs):
        try:
            from tensorflow.keras.utils import Sequence as TFSequence
            if TFSequence not in cls.__bases__:
                cls.__bases__ = cls.__bases__ + (TFSequence,)
        except ImportError:
            pass
        return super(DirectoryIterator, cls).__new__(cls)

    def __init__(self,
                 directory,
                 filter,
                 image_data_generator,
                 target_size=(256, 256),
                 color_mode='rgb',
                 classes=None,
                 class_mode='categorical',
                 batch_size=32,
                 shuffle=True,
                 seed=None,
                 data_format='channels_last',
                 save_to_dir=None,
                 save_prefix='',
                 save_format='png',
                 follow_links=False,
                 subset=None,
                 interpolation='nearest',
                 keep_aspect_ratio=False,
                 dtype='float32'):
        super(DirectoryIterator, self).set_processing_attrs(image_data_generator,
                                                            target_size,
                                                            color_mode,
                                                            data_format,
                                                            save_to_dir,
                                                            save_prefix,
                                                            save_format,
                                                            subset,
                                                            interpolation,
                                                            keep_aspect_ratio)
        
        
        self.directory = directory
        self.classes = classes
        if class_mode not in self.allowed_class_modes:
            raise ValueError('Invalid class_mode: {}; expected one of: {}'
                             .format(class_mode, self.allowed_class_modes))
        self.class_mode = class_mode
        self.dtype = dtype
        # First, count the number of samples and classes.
        self.samples = 0

        if not classes:
            classes = []
            for subdir in sorted(os.listdir(directory)):
                if os.path.isdir(os.path.join(directory, subdir)):
                    classes.append(subdir)
        self.num_classes = len(classes)
        self.class_indices = dict(zip(classes, range(len(classes))))

        pool = multiprocessing.pool.ThreadPool()

        # Second, build an index of the images
        # in the different class subfolders.
        results = []
        self.filenames = []
        i = 0
        for dirpath in (os.path.join(directory, subdir) for subdir in classes):
            results.append(
                pool.apply_async(_list_valid_filenames_in_directory,
                                 (dirpath, self.white_list_formats, self.split,
                                  self.class_indices, follow_links)))
        classes_list = []
        for res in results:
            classes, filenames = res.get()
            filtered_classes=[] 
            filtered_filenames=[] 
            i=0
            for filename in filtered_filenames:
                boolvalue=filter(filename)
                if(boolvalue==1):
                    filtered_classes.append(classes[i])
                    filtered_filenames.append(filename)
                i+=1
            classes_list.append(filtered_classes)
            self.filenames += filtered_filenames
        self.samples = len(self.filenames)
        self.classes = np.zeros((self.samples,), dtype='int32')
        for classes in classes_list:
            self.classes[i:i + len(classes)] = classes
            i += len(classes)

        print('Found %d images belonging to %d classes.' %
              (self.samples, self.num_classes))
        pool.close()
        pool.join()
        self._filepaths = [
            os.path.join(self.directory, fname) for fname in self.filenames
        ]
        super(DirectoryIterator, self).__init__(self.samples,
                                                batch_size,
                                                shuffle,
                                                seed)
    @property
    def filepaths(self):
        return self._filepaths

    @property
    def labels(self):
        return self.classes

    @property  # mixin needs this property to work
    def sample_weight(self):
        # no sample weights will be returned
        return None

list_valid_filenames_in_directory() wird verwendet, um alle Klassen und Dateinamen aus dem Verzeichnis zu extrahieren.

Original code: https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/image/utils.py#L218-L261



In [ ]:
def _list_valid_filenames_in_directory(directory, white_list_formats, split,
                                       class_indices, follow_links):
  
    """Lists paths of files in `subdir` with extensions in `white_list_formats`.
    # Arguments
        directory: absolute path to a directory containing the files to list.
                  The directory name is used as class label and must be a key of `class_indices`.
        white_list_formats: set of strings containing allowed extensions for
            the files to be counted.
        split: tuple of floats (e.g. `(0.2, 0.6)`) to only take into
            account a certain fraction of files in each directory.
            E.g.: `segment=(0.6, 1.0)` would only account for last 40 percent
            of images in each directory.
        class_indices: dictionary mapping a class name to its index.
        follow_links: boolean, follow symbolic links to subdirectories.
    # Returns
         classes: a list of class indices
         filenames: the path of valid files in `directory`, relative from
             `directory`'s parent (e.g., if `directory` is "dataset/class1",
            the filenames will be
            `["class1/file1.jpg", "class1/file2.jpg", ...]`).
    """
    dirname = os.path.basename(directory)
    if split:
        all_files = list(_iter_valid_files(directory, white_list_formats,follow_links))
        num_files = len(all_files)
        start, stop = int(split[0] * num_files), int(split[1] * num_files)
        valid_files = all_files[start: stop]
    else:
        valid_files = _iter_valid_files(
            directory, white_list_formats, follow_links)
    classes = []
    filenames = []
    for root, fname in valid_files:
        classes.append(class_indices[dirname])
        absolute_path = os.path.join(root, fname)
        relative_path = os.path.join(
            dirname, os.path.relpath(absolute_path, directory))
        filenames.append(relative_path)

    return classes, filenames